In [ ]:
# Install required packages
%pip install bertopic sentence-transformers umap-learn hdbscan gensim plotly scikit-learn -q

In [ ]:
# ============================================================
# PROJECT PATH SETUP
# ============================================================

import os
from pathlib import Path

if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/BERTopic-arXiv-Analysis'
    print("✅ Running on Google Colab")
else:
    PROJECT_PATH = str(Path(os.getcwd()).parent) if 'notebooks' in os.getcwd() else os.getcwd()
    print("✅ Running locally")

print(f"📁 Project path: {PROJECT_PATH}")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import json
import time
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
from collections import Counter

# BERTopic components
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# Evaluation
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.metrics import silhouette_score

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Libraries imported!")

In [ ]:
# Check GPU
import torch
if torch.cuda.is_available():
    device = 'cuda'
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = 'mps'
    print("✅ Apple Silicon GPU available")
else:
    device = 'cpu'
    print("⚠️ No GPU - using CPU (will be slower)")

## 1. Load Processed Data

In [ ]:
# Load processed data
df = pd.read_csv(f"{PROJECT_PATH}/data/processed/arxiv_cs_ai_processed.csv")
documents = df['text'].tolist()

print(f"Loaded {len(documents)} documents")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Prepare tokenized docs for coherence computation
tokenized_docs = [doc.lower().split() for doc in documents]
dictionary = Dictionary(tokenized_docs)
dictionary.filter_extremes(no_below=5, no_above=0.95)
print(f"Dictionary size: {len(dictionary)}")

## 2. Embedding Model Comparison

We compare two popular Sentence-BERT models:

| Model | Parameters | Speed | Quality |
|-------|------------|-------|--------|
| **all-mpnet-base-v2** | 110M | Slower | Higher |
| **all-MiniLM-L6-v2** | 22M | 5x faster | Good |

In [ ]:
# Define embedding models to compare
EMBEDDING_MODELS = {
    'mpnet': 'all-mpnet-base-v2',
    'minilm': 'all-MiniLM-L6-v2'
}

embeddings_dict = {}

for name, model_name in EMBEDDING_MODELS.items():
    embeddings_path = f"{PROJECT_PATH}/data/embeddings/embeddings_{name}.npy"
    
    if os.path.exists(embeddings_path):
        print(f"Loading pre-computed {name} embeddings...")
        embeddings_dict[name] = np.load(embeddings_path)
    else:
        print(f"\nComputing embeddings with {model_name}...")
        start_time = time.time()
        
        model = SentenceTransformer(model_name)
        embeddings = model.encode(
            documents,
            batch_size=64,
            show_progress_bar=True,
            convert_to_numpy=True,
            device=device
        )
        
        elapsed = time.time() - start_time
        print(f"  Time: {elapsed:.1f}s ({len(documents)/elapsed:.1f} docs/sec)")
        
        # Save embeddings
        np.save(embeddings_path, embeddings)
        embeddings_dict[name] = embeddings
        print(f"  Saved to {embeddings_path}")

print(f"\n✅ Embeddings ready:")
for name, emb in embeddings_dict.items():
    print(f"  {name}: {emb.shape}")

## 3. Evaluation Functions

In [ ]:
def compute_metrics(topic_model, documents, embeddings, topics, tokenized_docs, dictionary):
    """
    Compute all evaluation metrics for a BERTopic model.
    
    Returns:
        dict with coherence, diversity, silhouette, n_topics, outlier_pct
    """
    # Get topic words
    topics_dict = topic_model.get_topics()
    if -1 in topics_dict:
        del topics_dict[-1]
    
    n_topics = len(topics_dict)
    
    # Outlier percentage
    n_outliers = sum(1 for t in topics if t == -1)
    outlier_pct = 100 * n_outliers / len(topics)
    
    # Topic Diversity
    all_words = []
    topic_words_list = []
    for topic_id in sorted(topics_dict.keys()):
        words = [word for word, _ in topics_dict[topic_id][:10]]
        topic_words_list.append(words)
        all_words.extend(words)
    
    diversity = len(set(all_words)) / len(all_words) if all_words else 0
    
    # Coherence (NPMI)
    try:
        coherence_model = CoherenceModel(
            topics=topic_words_list,
            texts=tokenized_docs,
            dictionary=dictionary,
            coherence='c_npmi'
        )
        coherence = coherence_model.get_coherence()
    except:
        coherence = 0.0
    
    # Silhouette Score
    try:
        mask = np.array(topics) != -1
        if mask.sum() > 100 and len(set(np.array(topics)[mask])) > 1:
            silhouette = silhouette_score(embeddings[mask], np.array(topics)[mask])
        else:
            silhouette = 0.0
    except:
        silhouette = 0.0
    
    return {
        'n_topics': n_topics,
        'outlier_pct': outlier_pct,
        'coherence': coherence,
        'diversity': diversity,
        'silhouette': silhouette,
        'combined_score': coherence * 0.5 + diversity * 0.3 + (1 - outlier_pct/100) * 0.2
    }

print("Evaluation functions defined!")

## 4. Hyperparameter Grid Search

We'll search over:
- **Embedding model**: MPNet vs MiniLM
- **min_cluster_size**: [10, 15, 20, 30, 50]
- **n_neighbors**: [10, 15, 25]
- **n_components**: [5, 10]

In [ ]:
# Define hyperparameter grid
PARAM_GRID = {
    'embedding_model': ['mpnet', 'minilm'],
    'min_cluster_size': [10, 15, 20, 30, 50],
    'n_neighbors': [10, 15, 25],
    'n_components': [5, 10]
}

# Calculate total combinations
n_combinations = 1
for values in PARAM_GRID.values():
    n_combinations *= len(values)

print(f"Total hyperparameter combinations: {n_combinations}")
print(f"Estimated time: {n_combinations * 0.5:.0f} minutes")

In [ ]:
# Run grid search
results = []
best_score = -float('inf')
best_params = None
best_model = None

print("Starting hyperparameter search...\n")

for emb_name in PARAM_GRID['embedding_model']:
    embeddings = embeddings_dict[emb_name]
    
    for min_cluster_size in PARAM_GRID['min_cluster_size']:
        for n_neighbors in PARAM_GRID['n_neighbors']:
            for n_components in PARAM_GRID['n_components']:
                
                params = {
                    'embedding_model': emb_name,
                    'min_cluster_size': min_cluster_size,
                    'n_neighbors': n_neighbors,
                    'n_components': n_components
                }
                
                try:
                    # Build model
                    umap_model = UMAP(
                        n_neighbors=n_neighbors,
                        n_components=n_components,
                        min_dist=0.0,
                        metric='cosine',
                        random_state=42
                    )
                    
                    hdbscan_model = HDBSCAN(
                        min_cluster_size=min_cluster_size,
                        min_samples=10,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True
                    )
                    
                    vectorizer_model = CountVectorizer(
                        ngram_range=(1, 2),
                        stop_words='english',
                        min_df=5,
                        max_df=0.95
                    )
                    
                    topic_model = BERTopic(
                        umap_model=umap_model,
                        hdbscan_model=hdbscan_model,
                        vectorizer_model=vectorizer_model,
                        top_n_words=10,
                        calculate_probabilities=False,
                        verbose=False
                    )
                    
                    # Fit model
                    topics, _ = topic_model.fit_transform(documents, embeddings=embeddings)
                    
                    # Compute metrics
                    metrics = compute_metrics(
                        topic_model, documents, embeddings, topics,
                        tokenized_docs, dictionary
                    )
                    metrics.update(params)
                    results.append(metrics)
                    
                    # Check if best
                    if metrics['combined_score'] > best_score:
                        best_score = metrics['combined_score']
                        best_params = params.copy()
                        best_model = topic_model
                        best_topics = topics
                        best_embeddings = embeddings
                    
                    print(f"✓ {emb_name}, mcs={min_cluster_size}, nn={n_neighbors}, nc={n_components} → "
                          f"Topics: {metrics['n_topics']}, Coh: {metrics['coherence']:.3f}, "
                          f"Div: {metrics['diversity']:.3f}, Out: {metrics['outlier_pct']:.1f}%")
                    
                except Exception as e:
                    print(f"✗ {emb_name}, mcs={min_cluster_size}, nn={n_neighbors}, nc={n_components} → Error: {str(e)[:50]}")

print(f"\n{'='*60}")
print(f"GRID SEARCH COMPLETE!")
print(f"{'='*60}")

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('combined_score', ascending=False)

print("Top 10 Configurations:")
display_cols = ['embedding_model', 'min_cluster_size', 'n_neighbors', 'n_components',
                'n_topics', 'coherence', 'diversity', 'outlier_pct', 'combined_score']
print(results_df[display_cols].head(10).to_string(index=False))

# Save results
results_df.to_csv(f"{PROJECT_PATH}/results/hyperparameter_search_results.csv", index=False)
print(f"\nResults saved to hyperparameter_search_results.csv")

In [ ]:
# Visualize hyperparameter effects
fig = make_subplots(rows=2, cols=2, subplot_titles=[
    'Coherence by min_cluster_size',
    'Coherence by Embedding Model',
    'Topics vs Coherence',
    'Outlier % vs Coherence'
])

# 1. min_cluster_size effect
for emb in results_df['embedding_model'].unique():
    subset = results_df[results_df['embedding_model'] == emb]
    grouped = subset.groupby('min_cluster_size')['coherence'].mean()
    fig.add_trace(
        go.Scatter(x=grouped.index, y=grouped.values, name=emb, mode='lines+markers'),
        row=1, col=1
    )

# 2. Embedding model comparison
emb_comparison = results_df.groupby('embedding_model')['coherence'].agg(['mean', 'std'])
fig.add_trace(
    go.Bar(x=emb_comparison.index, y=emb_comparison['mean'], 
           error_y=dict(type='data', array=emb_comparison['std']),
           name='Coherence', showlegend=False),
    row=1, col=2
)

# 3. Topics vs Coherence
fig.add_trace(
    go.Scatter(x=results_df['n_topics'], y=results_df['coherence'],
               mode='markers', name='Configs', showlegend=False,
               marker=dict(color=results_df['combined_score'], colorscale='Viridis')),
    row=2, col=1
)

# 4. Outlier % vs Coherence
fig.add_trace(
    go.Scatter(x=results_df['outlier_pct'], y=results_df['coherence'],
               mode='markers', name='Configs', showlegend=False,
               marker=dict(color=results_df['combined_score'], colorscale='Viridis')),
    row=2, col=2
)

fig.update_layout(height=700, title_text="Hyperparameter Analysis")
fig.show()

fig.write_html(f"{PROJECT_PATH}/results/hyperparameter_analysis.html")

## 5. Best Model Analysis

In [ ]:
print("="*60)
print("BEST MODEL CONFIGURATION")
print("="*60)
print(f"\nEmbedding Model: {best_params['embedding_model']}")
print(f"min_cluster_size: {best_params['min_cluster_size']}")
print(f"n_neighbors: {best_params['n_neighbors']}")
print(f"n_components: {best_params['n_components']}")
print(f"\nCombined Score: {best_score:.4f}")

# Get best model metrics
best_metrics = results_df.iloc[0]
print(f"\nMetrics:")
print(f"  Topics: {int(best_metrics['n_topics'])}")
print(f"  Coherence (NPMI): {best_metrics['coherence']:.4f}")
print(f"  Diversity: {best_metrics['diversity']:.4f}")
print(f"  Outlier %: {best_metrics['outlier_pct']:.1f}%")
print(f"  Silhouette: {best_metrics['silhouette']:.4f}")

## 6. Outlier Reduction

BERTopic can reassign outliers to their nearest topics using various strategies.

In [ ]:
# Count outliers before
n_outliers_before = sum(1 for t in best_topics if t == -1)
print(f"Outliers before reduction: {n_outliers_before} ({100*n_outliers_before/len(best_topics):.1f}%)")

# Reduce outliers using c-TF-IDF strategy
print("\nReducing outliers using c-TF-IDF similarity...")
new_topics = best_model.reduce_outliers(
    documents, 
    best_topics,
    strategy="c-tf-idf",
    threshold=0.1
)

# Update model with new topics
best_model.update_topics(documents, topics=new_topics)

# Count outliers after
n_outliers_after = sum(1 for t in new_topics if t == -1)
print(f"Outliers after reduction: {n_outliers_after} ({100*n_outliers_after/len(new_topics):.1f}%)")
print(f"Outliers reduced by: {n_outliers_before - n_outliers_after} documents")

In [ ]:
# Recompute metrics after outlier reduction
final_metrics = compute_metrics(
    best_model, documents, best_embeddings, new_topics,
    tokenized_docs, dictionary
)

print("\nMetrics After Outlier Reduction:")
print(f"  Topics: {final_metrics['n_topics']}")
print(f"  Coherence (NPMI): {final_metrics['coherence']:.4f}")
print(f"  Diversity: {final_metrics['diversity']:.4f}")
print(f"  Outlier %: {final_metrics['outlier_pct']:.1f}%")
print(f"  Silhouette: {final_metrics['silhouette']:.4f}")

## 7. Embedding Model Comparison Summary

In [ ]:
# Compare embedding models
emb_summary = results_df.groupby('embedding_model').agg({
    'coherence': ['mean', 'std', 'max'],
    'diversity': ['mean', 'max'],
    'n_topics': ['mean', 'min', 'max'],
    'outlier_pct': ['mean', 'min']
}).round(4)

print("\n" + "="*60)
print("EMBEDDING MODEL COMPARISON")
print("="*60)
print(emb_summary.to_string())

# Winner determination
mpnet_best = results_df[results_df['embedding_model'] == 'mpnet']['coherence'].max()
minilm_best = results_df[results_df['embedding_model'] == 'minilm']['coherence'].max()

print(f"\n🏆 Best MPNet coherence: {mpnet_best:.4f}")
print(f"🏆 Best MiniLM coherence: {minilm_best:.4f}")
print(f"\n{'MPNet' if mpnet_best > minilm_best else 'MiniLM'} wins by {abs(mpnet_best - minilm_best):.4f}!")

In [ ]:
# Visualization: MPNet vs MiniLM
fig = go.Figure()

for emb in ['mpnet', 'minilm']:
    subset = results_df[results_df['embedding_model'] == emb]
    fig.add_trace(go.Box(
        y=subset['coherence'],
        name=emb.upper(),
        boxpoints='all',
        jitter=0.3
    ))

fig.update_layout(
    title='Coherence Score Distribution: MPNet vs MiniLM',
    yaxis_title='Coherence (NPMI)',
    showlegend=False
)
fig.show()

fig.write_html(f"{PROJECT_PATH}/results/embedding_model_comparison.html")

## 8. Save Best Model

In [ ]:
# Save best model
best_model_path = f"{PROJECT_PATH}/models/bertopic_best_model"
os.makedirs(best_model_path, exist_ok=True)

best_model.save(
    best_model_path,
    serialization="safetensors",
    save_ctfidf=True,
    save_embedding_model=False
)

print(f"Best model saved to {best_model_path}")

# Save best configuration
best_config = {
    'params': best_params,
    'metrics': {
        'n_topics': int(final_metrics['n_topics']),
        'coherence_npmi': float(final_metrics['coherence']),
        'diversity': float(final_metrics['diversity']),
        'outlier_pct': float(final_metrics['outlier_pct']),
        'silhouette': float(final_metrics['silhouette'])
    },
    'outlier_reduction': {
        'before': n_outliers_before,
        'after': n_outliers_after,
        'strategy': 'c-tf-idf'
    }
}

with open(f"{PROJECT_PATH}/results/best_config.json", 'w') as f:
    json.dump(best_config, f, indent=2)

print("Configuration saved to best_config.json")

In [ ]:
# Save topic assignments with best model
results_df_final = df.copy()
results_df_final['topic'] = new_topics

# Add topic labels
topic_info = best_model.get_topic_info()
topic_labels = {row['Topic']: row['Name'] for _, row in topic_info.iterrows()}
results_df_final['topic_name'] = results_df_final['topic'].map(topic_labels)

results_df_final.to_csv(f"{PROJECT_PATH}/results/topic_assignments_best.csv", index=False)
topic_info.to_csv(f"{PROJECT_PATH}/results/topic_info_best.csv", index=False)

print("Topic assignments saved!")

In [ ]:
# Compute and save 2D embeddings for best model
print("Computing 2D UMAP projection...")
umap_2d = UMAP(n_components=2, min_dist=0.1, metric='cosine', random_state=42)
embeddings_2d = umap_2d.fit_transform(best_embeddings)

np.save(f"{PROJECT_PATH}/data/embeddings/embeddings_2d_best.npy", embeddings_2d)
print("2D embeddings saved!")

## 9. Summary Report

In [ ]:
# Generate comprehensive report
report = f"""
{'='*70}
HYPERPARAMETER TUNING & MODEL COMPARISON REPORT
Project: Topic Modeling arXiv cs.AI with BERTopic
Team: Pavan Chauhan, Vedanta Nayak
{'='*70}

HYPERPARAMETER SEARCH
{'-'*50}
Total configurations tested: {len(results_df)}
Embedding models: MPNet, MiniLM
min_cluster_size: [10, 15, 20, 30, 50]
n_neighbors: [10, 15, 25]
n_components: [5, 10]

EMBEDDING MODEL COMPARISON
{'-'*50}
MPNet (all-mpnet-base-v2):
  - Best coherence: {results_df[results_df['embedding_model']=='mpnet']['coherence'].max():.4f}
  - Mean coherence: {results_df[results_df['embedding_model']=='mpnet']['coherence'].mean():.4f}
  - Parameters: 110M

MiniLM (all-MiniLM-L6-v2):
  - Best coherence: {results_df[results_df['embedding_model']=='minilm']['coherence'].max():.4f}
  - Mean coherence: {results_df[results_df['embedding_model']=='minilm']['coherence'].mean():.4f}
  - Parameters: 22M (5x smaller)

BEST MODEL
{'-'*50}
Embedding: {best_params['embedding_model'].upper()}
min_cluster_size: {best_params['min_cluster_size']}
n_neighbors: {best_params['n_neighbors']}
n_components: {best_params['n_components']}

OUTLIER REDUCTION
{'-'*50}
Strategy: c-TF-IDF similarity
Before: {n_outliers_before} outliers ({100*n_outliers_before/len(documents):.1f}%)
After: {n_outliers_after} outliers ({100*n_outliers_after/len(documents):.1f}%)
Improvement: {n_outliers_before - n_outliers_after} documents reassigned

FINAL METRICS
{'-'*50}
Topics: {final_metrics['n_topics']}
Coherence (NPMI): {final_metrics['coherence']:.4f}
Topic Diversity: {final_metrics['diversity']:.4f} ({final_metrics['diversity']*100:.1f}%)
Silhouette Score: {final_metrics['silhouette']:.4f}
Outlier %: {final_metrics['outlier_pct']:.1f}%

{'='*70}
"""

print(report)

with open(f"{PROJECT_PATH}/results/hyperparameter_tuning_report.txt", 'w') as f:
    f.write(report)

print(f"Report saved!")

## Summary

This notebook has:
1. ✅ Compared MPNet vs MiniLM embeddings
2. ✅ Performed grid search over 60 hyperparameter combinations
3. ✅ Identified optimal configuration
4. ✅ Applied outlier reduction
5. ✅ Saved best model and results

**Next step:** Run `04_evaluation.ipynb` for full evaluation with LDA baseline comparison.